# Model Development on Mobile Images


In [58]:
# import libraries for data processing
import numpy as np
import pandas as pd

# import libraries for image processing and system configuration
import io
import requests
import cv2
from PIL import Image

In [59]:
# import tensorflow
import tensorflow as tf

In [74]:
# import tensorflow for building neural network
from tensorflow import keras
from keras import models
from keras import layers
# from keras.optimizers import 

### Model Development Approach

+ Image Data Preparation
+ Model Building
+ Model Training
+ Model Evaluation

## Image Data Preparation

**Image Processing 3D Arrays**
+ download images and convert into 3D Tensors
+ batch multiple images together
+ processing images using tensorflow utilities

In [61]:
# load image data
images_df = pd.read_csv("mobile_phone_images.csv", index_col=0)
images_df.head(3)

,Image_URL,ImageType
0,https://www.orange.lu/media/catalog/product/i/...,Iphone
1,https://www.orange.lu/media/catalog/product/i/...,Iphone
2,https://www.orange.lu/media/catalog/product/i/...,Iphone


In [62]:
# download images and convert into numpy arrays

def convert_images_to_arrays(img_url, target_size=(32,32)):
  response = requests.get(img_url) # download the image

   # open + resize the image using the target size
  img = Image.open(io.BytesIO(response.content)) # open the image
  img = img.resize(target_size)
  img = img.convert('RGB')
  
  # convert into array + normalize the image 
  img_array = np.array(img) 
  img_array = img_array.astype('float32')/225.0

  return img_array

test


In [63]:
# create instances to store 3d image arrays from img urls
array_3d_coll = []
img_index_list = images_df.index

# iterate the image url + add into a list of all 3D-arrays
for i in images_df.index: 
  img_url = images_df["Image_URL"].iloc[i]
  img_array = convert_images_to_arrays(img_url)
  array_3d_coll.append(img_array)

# create a new feature --> containing all 3d arrays + add into dataframe 
images_df["Image_Array3D"] = array_3d_coll

In [64]:
# print the shape of the image 3D array
img_array3D = images_df["Image_Array3D"].iloc[0]
img3D_shape = img_array3D.shape
print(f"Image Array 3D - Shape: {img3D_shape}")

Image Array 3D - Shape: (32, 32, 3)


In [65]:
# display dataframe
images_df.head()

,Image_URL,ImageType,Image_Array3D
0,https://www.orange.lu/media/catalog/product/i/...,Iphone,"[[[1.1333333, 1.1333333, 1.1333333], [1.133333..."
1,https://www.orange.lu/media/catalog/product/i/...,Iphone,"[[[1.1333333, 1.1333333, 1.1333333], [1.133333..."
2,https://www.orange.lu/media/catalog/product/i/...,Iphone,"[[[1.1333333, 1.1333333, 1.1333333], [1.133333..."
3,https://www.orange.lu/media/catalog/product/i/...,Iphone,"[[[1.1333333, 1.1333333, 1.1333333], [1.133333..."
4,https://www.orange.lu/media/catalog/product/s/...,Samsung,"[[[1.1333333, 1.1333333, 1.1333333], [1.133333..."


## Model Training

Procedure for building image classification model to train and develop the model on mobile images using mobile labels:
+ prepare training samples and labels for model training
+ build a convolutional neural network from TensorFlow
+ train the model on mobile images with 10 epochs
+ evaluate the images + 
+ graphically analyse the loss and the accuracy of the training process

In [66]:
# extract data from image dataframe
img_features = ["Image_Array3D", "ImageType"]
mobile_img_data = images_df[img_features]

In [67]:
img = mobile_img_data["Image_Array3D"].loc[13]
img

array([[[1.1333333 , 1.1333333 , 1.1333333 ],
        [1.0888889 , 1.0888889 , 1.0888889 ],
        [0.6666667 , 0.6666667 , 0.6755555 ],
        ...,
        [0.6577778 , 0.6533333 , 0.6666667 ],
        [1.0888889 , 1.0888889 , 1.0888889 ],
        [1.1333333 , 1.1333333 , 1.1333333 ]],

       [[1.1333333 , 1.1333333 , 1.1333333 ],
        [0.69777775, 0.69777775, 0.70666665],
        [0.13777778, 0.13777778, 0.16444445],
        ...,
        [0.13777778, 0.13777778, 0.16888888],
        [0.6888889 , 0.6888889 , 0.7022222 ],
        [1.1333333 , 1.1333333 , 1.1333333 ]],

       [[1.1333333 , 1.1333333 , 1.1333333 ],
        [0.5733333 , 0.5733333 , 0.5822222 ],
        [0.16444445, 0.16      , 0.19111112],
        ...,
        [0.16888888, 0.16888888, 0.2       ],
        [0.52      , 0.52444446, 0.5377778 ],
        [1.1333333 , 1.1333333 , 1.1333333 ]],

       ...,

       [[1.1333333 , 1.1333333 , 1.1333333 ],
        [0.5422222 , 0.5422222 , 0.5511111 ],
        [0.31111112, 0

In [68]:
# rename 
mobile_img_data = mobile_img_data.rename(columns={
  "Image_Array3D": "Images", 
  "ImageType": "Image_Labels"
})

In [69]:
mobile_img_data.head(2)

,Images,Image_Labels
0,"[[[1.1333333, 1.1333333, 1.1333333], [1.133333...",Iphone
1,"[[[1.1333333, 1.1333333, 1.1333333], [1.133333...",Iphone


In [88]:
# prepare the image labels and samples
train_mobile_images = mobile_img_data["Images"].sample(frac=0.8)
train_labels = mobile_img_data["Image_Labels"].sample(frac=0.8)

test_mobile_images = mobile_img_data[""]

In [89]:
# show numbers of labels
labels = train_labels.unique()
mobile_labels = len(labels)
print(f"Number of mobile brand labels: {mobile_labels}")

Number of mobile brand labels: 5


In [86]:
# build convolutional neural network model 
def convolutional_neural_network(num_labels): 
  # define a cnn model
  model = models.Sequential()

  # add convolutional layer, followed by a max-pooling layers
  model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,3)))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(64, (3,3), activation="relu"))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(64, (3,3), activation="relu"))
  
  # flatten 3D features into 1D to connect dense layers
  model.add(layers.Flatten())

  # add dense layers
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dense(num_labels))

  # compile the model 
  model.compile(optimizer="adam",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
  return model 

In [87]:
# build a convolutional neural network model
cnn_model = convolutional_neural_network(mobile_labels)

In [ ]:
# train model
model_history = cnn_model.fit(train)